## Load Data

In [17]:
import pandas as pd
import numpy as np
import glob
from pyntcloud import PyntCloud

In [18]:
def load_ply(file_name):
    cloud = PyntCloud.from_file(file_name)
    return cloud.points.values

In [19]:
def load_obj(file_name):
    vertices = []
    with open(file_name) as f:
        for line in f:
            if line[:2] == 'v ':
                index1 = line.find(' ') + 1
                index2 = line.find(' ', index1 + 1)
                index3 = line.find(' ', index2 + 1)

                vertex = (float(line[index1:index2]), float(line[index2:index3]), float(line[index3:-1]))
                vertex = [round(vertex[0], 2), round(vertex[1], 2), round(vertex[2], 2)]
                vertices.append(np.array(vertex))
                
    return np.array(vertices)

In [20]:
def load_off(file_name):
    vertices = []
    with open(file_name) as f:
        for i, line in enumerate(f):
            vals = line.split(' ')
            if i > 2 and len(vals) == 3:
                vertex = [float(vals[0]), float(vals[1]), float(vals[2])]
                vertices.append(np.array(vertex))

    return np.random.permutation(np.array(vertices))[:2048]

## Create Dataset

In [21]:
from torch.utils.data import Dataset, DataLoader

In [22]:
class PointCloudDataset(Dataset):
    """Point cloud dataset."""

    def __init__(self, number=-1, directory='./data/04379243'):
        
        file_names = glob.glob('%s/*.ply' % directory)

        if number > 0 and len(file_names) > number:
            file_names = file_names[:number]
        
        point_clouds = []
        for file_name in file_names:

            points = load_ply(file_name)
            point_clouds.append(points)

        self.point_clouds = np.array(point_clouds, dtype='float64')
        self.point_clouds = np.transpose(self.point_clouds, (0, 2, 1))
        
    def __len__(self):
        return len(self.point_clouds)

    def __getitem__(self, idx):

        return self.point_clouds[idx]

In [23]:
point_cloud_dataset = PointCloudDataset(200)
dataloader = DataLoader(point_cloud_dataset, batch_size=20, shuffle=True)

## Auto encoder

In [16]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np

In [25]:
class Encoder(nn.Module):

    def __init__(self):
        super(Encoder, self).__init__()
        
        self.conv1 = nn.Conv1d(3, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 256, 1)
        self.conv4 = nn.Conv1d(256, 256, 1)
        self.conv5 = nn.Conv1d(256, 128, 1)
        self.maxPool1d = nn.MaxPool1d(2048)
        
        self.relu = nn.ReLU()
        self.bn1  = nn.BatchNorm1d(64) 
        self.bn2  = nn.BatchNorm1d(128) 
        self.bn3  = nn.BatchNorm1d(256)
        self.bn4  = nn.BatchNorm1d(256) 
        self.bn5  = nn.BatchNorm1d(128) 

    def forward(self, x):

        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.relu(self.bn5(self.conv5(x)))
        x = self.maxPool1d(x)
        return x

Encoder()

Encoder(
  (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
  (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
  (conv3): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
  (conv4): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
  (conv5): Conv1d(256, 128, kernel_size=(1,), stride=(1,))
  (maxPool1d): MaxPool1d(kernel_size=2048, stride=2048, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [26]:
class Decoder(nn.Module):

    def __init__(self):
        super(Decoder, self).__init__()

        self.fc1 = nn.Linear(128, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 2048*3)

        self.relu = nn.ReLU()
        self.bn1  = nn.BatchNorm1d(256) 
        self.bn2  = nn.BatchNorm1d(256) 
        
    def forward(self, x):

        x = x.view(x.shape[0], -1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        
        x = x.view(-1, 3, 2048)
        
        return x
    
Decoder()

Decoder(
  (fc1): Linear(in_features=128, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=6144, bias=True)
  (relu): ReLU()
  (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [27]:
class AutoEncoder(nn.Module):
    
    def __init__(self):
        super(AutoEncoder, self).__init__()

        self.encoder = Encoder()
        self.decoder = Decoder()

    def forward(self, x):

        x = self.encoder(x)
        x = self.decoder(x)
        
        return x

## Training

In [34]:
from lib.chamfer import ChamferLoss

model = AutoEncoder()

criterion = ChamferLoss()

num_epochs = 2000
learning_rate = 1e-2

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

if torch.cuda.is_available:
    model = model.cuda()

for epoch in range(num_epochs):

    for data in dataloader:

        data = data.float()
        
        if torch.cuda.is_available:
            data = data.cuda()
            
        train_output = model(data)
        loss = criterion(train_output, data)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch + 1) % 30 == 0:
        print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss.data.mean()))

epoch [30/2000], loss:4392.4868
epoch [60/2000], loss:3955.4326
epoch [90/2000], loss:3893.6924
epoch [120/2000], loss:3883.5156
epoch [150/2000], loss:3938.4954
epoch [180/2000], loss:3996.1606
epoch [210/2000], loss:3765.2651
epoch [240/2000], loss:4019.3213
epoch [270/2000], loss:3817.7510
epoch [300/2000], loss:4155.7900
epoch [330/2000], loss:3767.2783
epoch [360/2000], loss:3387.7898
epoch [390/2000], loss:4136.5718
epoch [420/2000], loss:3715.8105
epoch [450/2000], loss:3861.6033
epoch [480/2000], loss:3774.4512
epoch [510/2000], loss:3724.6223
epoch [540/2000], loss:3850.7087
epoch [570/2000], loss:3636.8374
epoch [600/2000], loss:3554.3472
epoch [630/2000], loss:3728.0703
epoch [660/2000], loss:4132.7754
epoch [690/2000], loss:3646.8154
epoch [720/2000], loss:3652.8970
epoch [750/2000], loss:3502.6401
epoch [780/2000], loss:3624.4683
epoch [810/2000], loss:3508.3906
epoch [840/2000], loss:3527.9639
epoch [870/2000], loss:3849.3491
epoch [900/2000], loss:3887.4478
epoch [930/20

KeyboardInterrupt: 

#### save

In [ ]:
torch.save(model.encoder.state_dict(), './models/encoder.pt')
torch.save(model.decoder.state_dict(), './models/decoder.pt')

### get input values for validation

In [ ]:
train_input  = next(iter(dataloader)).float()

if torch.cuda.is_available :
    train_output = model(train_input.cuda())
    train_output = train_output.cpu()
else:
    train_output = model(train_input)

In [ ]:
v_encoder = Encoder()
v_decoder = Decoder()

v_encoder.load_state_dict(torch.load('./models/encoder.pt'))
v_decoder.load_state_dict(torch.load('./models/decoder.pt'))

## Generator

In [ ]:
class Generator(nn.Module):
    
    def __init__(self, encoder, decoder):
        super(Generator, self).__init__()

        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x):

        x = self.encoder(x)
        x[:,1,0] *= 10.0
        x = self.decoder(x)
        
        return x

In [ ]:
generator = Generator(v_encoder, v_decoder)
generator_output = generator(train_input)

## Validation

In [36]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
fig = plt.figure(figsize=(18, 8))

for i in range(8):
    
    row, columns, num = 2, 4, i + 1
    ax = fig.add_subplot(row, columns, num, projection='3d')

    data = train_input[i].detach().numpy()
    ax.scatter(data[0], data[1], data[2], zdir='z', s=4, c='b', label='ground truth')

    data = train_output[i].detach().numpy()
    ax.scatter(data[0], data[1], data[2], zdir='z', s=4, c='r', label='decoded')

    data = generator_output[i].detach().numpy()
    ax.scatter(data[0], data[1], data[2], zdir='z', s=4, c='g', label='decoded & modified')

ax.legend()
plt.show()

## Write to File

In [35]:
def write(points, file_name, directory='./output/'):
    
    if points.shape[0] < points.shape[1] and points.shape[0] == 3:
        points = points.T
    
    with open(directory + file_name, 'w') as f:
        
        f.write(str(points.shape[0]) + '\n')

        for pt in points:
            f.write('%.8f %.8f %.8f\n' % tuple(pt))
    
write(next(iter(dataloader))[2].detach().numpy(), 'point_cloud.txt')

## TODOs

* Manimulate latent vector or try something else to generate more nice tables
* Reduce the number of point clouds or connect each points reasonably

## Misc

In [ ]:
input = torch.randn(10, 2048, 3)
print(input.shape)
m = nn.Conv1d(2048, 256, 1)
output = m(input)
print(output.shape)
m = nn.Conv1d(256, 128, 1)
output = m(output)
m = nn.Conv1d(128, 64, 1)
output = m(output)
print(output.shape)
m = nn.Conv1d(64, 32, 1)
output = m(output)
print(output.shape)
m = nn.MaxPool1d(3)
output = m(output)
print(output.shape)

In [ ]:
input = torch.randn(10, 3, 2048)
print(input.shape)
m = nn.Conv1d(3, 64, 1)
output = m(input)
print(output.shape)
m = nn.Conv1d(64, 128, 1)
output = m(output)
m = nn.Conv1d(128, 256, 1)
output = m(output)
print(output.shape)
m = nn.Conv1d(256, 512, 1)
output = m(output)
print(output.shape)
m = nn.MaxPool1d(2048) # output = torch.max(output, 2, keepdim=True)[0]
output = m(output)
print(output.shape)

In [ ]:
input = Variable(torch.rand(32,3,2500))
print(input.shape)
m = torch.nn.Conv1d(3, 64, 1)
output = m(input)
print(output.shape)
